In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from nplinker_constants import nplinker_setup
nplinker_setup(LDA_PATH='/home/simon/git/lda/code')

In [30]:
from metabolomics import load_spectra
mgf_file = '/srv/data/crusemann/gnps/METABOLOMICS-SNETS-c36f90ba-download_clustered_spectra-main.mgf'
spectra = load_spectra(mgf_file)

Loaded 5930 molecules


In [34]:
from metabolomics import load_metadata
load_metadata(spectra,nodes_file)

In [35]:
for s in spectra:
    if len(s.annotations) > 0:
        print s.annotations


[('Actinomycin_X2;_Actinomycin_V;_Actinomyc_28731_dereplictor_pv_1.70529e-21', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_For

In [6]:
from metabolomics import load_edges
edge_file = '/srv/data/crusemann/gnps/9a93d720f69143bb9f971db39b5d2ba2.pairsinfo'
load_edges(spectra,edge_file)
from metabolomics import make_families
families = make_families(spectra)

In [88]:
import os,glob
from genomics import loadBGC_from_cluster_files
input_files = []
ann_files = []
# this is really messy - change folders to change which folers are included
# we make two lists of files - the clustering ones and the annotation ones
# clusterin gives us the mol families, networking the metadata (bigscape class etc)
# it's also really slow, because of the antismash file matching....which is a mess
# it works for this data but is incredibly non robust
root_path = '/srv/data/crusemann/bigscape/bigscape_corason_crusemann_complete_annotated_mibigs_mix_automode_20180713/network_files/2018-07-13_16-34-11_hybrids_auto_crusemann_bgcs_automode_mix_mibig'
folders = ['NRPS','Others','PKSI','PKS-NRP_Hybrids','PKSother','RiPPs','Saccharides','Terpene']
# ditched 'mix' and 'others'
for folder in folders:
    fam_file = os.path.join(root_path,folder)
    cluster_file = glob.glob(fam_file + os.sep + folder + "_clustering*")
    annotation_files = glob.glob(fam_file + os.sep + "Network_*")
    input_files.append(cluster_file[0])
    ann_files.append(annotation_files[0])
    
gcf_list,bgc_list, strain_list = loadBGC_from_cluster_files(input_files,ann_files,antismash_dir='/srv/data/crusemann/antismash/justin-20181022/',antismash_format = 'flat')

Next cell adds a random object to each spectrum and gcf object, for doing the permutation testing

In [114]:
# initiate the random ones
for g in gcf_list:
    g.add_random(strain_list)
for s in spectra:
    s.add_random(strain_list)

In [16]:
from scoring import compute_all_scores,metcalf_scoring
m_scores = compute_all_scores(spectra,gcf_list,strain_list,metcalf_scoring,do_random = True)

Done 0 of 5930
Best:  110
Best random:  110
Best:  128
Best random:  125
Best:  131
Best random:  126
Best:  134
Best random:  128
Best random:  131
Best:  138
Best:  144
Best random:  143
Best:  146
Best random:  146
Best:  148
Best random:  152
Best:  153
Best random:  153
Best random:  158
Best random:  161
Best random:  169
Best:  158
Best:  159
Best:  161
Best:  162
Best:  165
Done 100 of 5930
Best:  166
Done 200 of 5930
Best:  175
Done 300 of 5930
Best:  176
Done 400 of 5930
Done 500 of 5930
Best:  199
Best:  207
Best:  251
Best:  287
Done 600 of 5930
Done 700 of 5930
Done 800 of 5930
Done 900 of 5930
Done 1000 of 5930
Done 1100 of 5930
Done 1200 of 5930
Done 1300 of 5930
Done 1400 of 5930
Done 1500 of 5930
Done 1600 of 5930
Done 1700 of 5930
Done 1800 of 5930
Done 1900 of 5930
Done 2000 of 5930
Done 2100 of 5930
Done 2200 of 5930
Done 2300 of 5930
Done 2400 of 5930
Done 2500 of 5930
Done 2600 of 5930
Done 2700 of 5930
Done 2800 of 5930
Done 2900 of 5930
Done 3000 of 5930
Best:  

In [92]:
gcf_with_mibig = []
for gcf in gcf_list:
    if len(gcf.get_mibig_bgcs()) > 0:
        gcf_with_mibig.append(gcf)
print len(gcf_with_mibig),len(gcf_list)

43 1574


In [93]:
spectra_with_lib = []
for s in spectra:
    if len(s.annotations) > 0:
        spectra_with_lib.append(s)
print len(spectra_with_lib)

109


In [116]:
name_scores = {}
from scoring import name_scoring
name_scores = compute_all_scores(spectra_with_lib,gcf_with_mibig,mibig_map,name_scoring,do_random = False)
for s in spectra_with_lib:
    for g in gcf_with_mibig:
        if name_scores[s][g][0] > 0:
            print s,g,name_scores[s][g]
            sc,m = metcalf_scoring(s,g,strain_list)
            print len(g.random_gcf.strain_set),len(s.random_spectrum.strain_set)
            for st in m:
                print st.name

Done 0 of 109
Staurosporine Staurosporine M+H
Best:  100
Staurosporine Staurosporine M+H
Done 100 of 109
Spectrum 41296 with 16 peaks, max_ms2_intensity 66689.96875 <genomics.GCF object at 0x7f5f2ba7ca50> (100, None, ('Staurosporine', 'Staurosporine M+H'))
34 46
CNT005
CNS051
CNY011
CNY231
CNH718
CNY694
CNP193
CNS991
CNS296
CNX814
CNY237
CNY281
CNS744
CNY256
CNQ884
CNY234
CNH877
CNR107
CNB458
CNS325
CNY282
CNY280
CNT849
CNP105
CNY260
CNS342
CNR921
CNT088
CNH962
CNS673
CNT798
Spectrum 41297 with 32 peaks, max_ms2_intensity 7503.780762 <genomics.GCF object at 0x7f5f2ba7ca50> (100, None, ('Staurosporine', 'Staurosporine M+H'))
34 17
CNR107
CNR921
CNY260
CNB458
CNY282
CNS296
CNT849
CNS991


In [ ]:
from genomics import load_mibig_map
mibig_map = load_mibig_map()


In [15]:
import pylab as plt
import numpy as np
%matplotlib inline
scores =[]
random_scores = []
for s,g in m_scores.items():
    for gg,sc in g.items():
        random_scores.append(sc[1])
        scores.append(sc[0])
plt.hist(random_scores,bins=50)
plt.xlim([100,150])
plt.figure()
plt.hist(scores,bins=50)
plt.xlim([100,150])
plt.ylim([0,1000])
random_scores.sort()
perc99 = random_scores[int(np.floor(0.99*len(random_scores)))]
print perc99
scores = np.array(scores)
n_above = len(np.where(scores>=perc99)[0])
print (1.0*n_above)/len(scores)

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
best_spec = None
best_gcf = None
for spectrum in m_scores:
    for gcf in m_scores[spectrum]:
        if m_scores[spectrum][gcf] == best:
            best_spec = spectrum
            best_gcf = gcf

In [ ]:
print best_spec
print best_gcf
for strain in strain_list:
    print strain,best_spec.has_strain(strain),best_gcf.has_strain(strain)

In [ ]:
print best_spec.metadata['LibraryID']
for bgc in best_gcf.bgc_list:
    print bgc,bgc.bigscape_class,bgc.product_prediction

In [78]:
link_file = 'mibig_gnps_links_q3_loose.csv'
links = []
import csv
with open(link_file,'r') as f:
    reader = csv.reader(f)
    heads = reader.next()
    print heads
    for line in reader:
        links.append(line)

['#mibig_id', 'gnps_id', 'mibig_name', 'gnps_name']


In [98]:
link_dict = {}
for link in links:
    short_gnps_name = link[3].split()[0]
    link_dict[short_gnps_name] = link[0]    

In [102]:
spec_lib_dict = {}
for spectrum in spectra:
    if spectrum.metadata['LibraryID']:
        lid = spectrum.metadata['LibraryID'].split()[0]
        if lid in link_dict:
            spec_lib_dict[spectrum] = lid

In [107]:
for spec in spec_lib_dict:
    bgc_name = link_dict[spec_lib_dict[spec]]
    print bgc_name
    bgc = [b for b in bgc_list if b.name.startswith(bgc_name)]
    print bgc

BGC0000198
[]
BGC0001830
[]
BGC0000136
[]
BGC0001766
[]
BGC0001766
[]
BGC0001572
[]
BGC0000241
[]
BGC0000136
[]
BGC0001572
[]
BGC0000426
[]
BGC0000826
[]
BGC0000198
[]
BGC0000333
[]
BGC0001830
[]
BGC0001766
[]
BGC0000241
[]
BGC0000220
[]
BGC0000241
[]
BGC0000198
[]
BGC0000826
[]
BGC0001572
[]
BGC0001572
[]
BGC0000136
[]


In [106]:
print [b.name for b in bgc_list if b.name.startswith('BGC')]

['BGC0000181.1', 'BGC0001383.1', 'BGC0001031.1', 'BGC0001032.1', 'BGC0000995.1', 'BGC0001113.1', 'BGC0000418.1', 'BGC0000419.1', 'BGC0000995.1', 'BGC0001113.1', 'BGC0000064.1', 'BGC0001268.1', 'BGC0001096.1', 'BGC0000976.1', 'BGC0001165.1', 'BGC0000989.1', 'BGC0000990.1', 'BGC0001041.1', 'BGC0000991.1', 'BGC0001009.1', 'BGC0001010.1', 'BGC0000689.1', 'BGC0000697.1', 'BGC0001241.1', 'BGC0001267.1', 'BGC0001322.1', 'BGC0001324.1', 'BGC0000611.1', 'BGC0000655.1', 'BGC0000611.1', 'BGC0000655.1', 'BGC0000564.1', 'BGC0001170.1', 'BGC0000491.1', 'BGC0001222.1', 'BGC0000530.1', 'BGC0000533.1', 'BGC0000064.1', 'BGC0001268.1', 'BGC0000989.1', 'BGC0000991.1', 'BGC0000157.1', 'BGC0000158.1', 'BGC0001009.1', 'BGC0001010.1', 'BGC0001041.1', 'BGC0000976.1', 'BGC0001165.1', 'BGC0000958.1', 'BGC0001216.1', 'BGC0001340.1', 'BGC0001385.1', 'BGC0000004.1', 'BGC0000008.1', 'BGC0001259.1', 'BGC0001272.1', 'BGC0001264.1', 'BGC0001280.1', 'BGC0000006.1', 'BGC0000076.1', 'BGC0000077.1', 'BGC0000979.1', 'BGC000

In [125]:
from genomics_utilities import get_known_cluster_blast
for b in bgc_list:
    get_known_cluster_blast(b)

In [126]:
for b in bgc_list:
    if hasattr(b,'metadata'):
        kcb = b.metadata.get('knownclusterblast',None)
        if kcb:
            print kcb
            break

[('BGC0001283_c1', '83'), ('BGC0000291_c1', '20'), ('BGC0000354_c1', '12'), ('BGC0000379_c1', '12'), ('BGC0001406_c1', '8'), ('BGC0000315_c1', '16'), ('BGC0001370_c1', '16'), ('BGC0000439_c1', '16'), ('BGC0001368_c1', '8'), ('BGC0000336_c1', '16')]


In [133]:
from scoring import knownclusterblast_scoring
kcb_scores = compute_all_scores(spectra_with_lib,gcf_list,mibig_map,knownclusterblast_scoring,do_random = False)

Done 0 of 109
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Best:  3
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Sta

Enterocin Enterocin M+H
('Enterocin', 'Enterocin M+H')
Enterocin Enterocin M+H
('Enterocin', 'Enterocin M+H')
Enterocin Enterocin M+H
('Enterocin', 'Enterocin M+H')
Done 100 of 109


In [137]:
for s in kcb_scores:
    for g in kcb_scores[s]:
        sc,_,metadata = kcb_scores[s][g]
        if sc > 0:
            a,_ = metcalf_scoring(s,g,strain_list)
            print sc,metadata,a

125 ('Enterocin', 'Enterocin M+H') 131
95 ('Enterocin', 'Enterocin M+H') 153
2196 ('Staurosporine', 'Staurosporine M+H') -368
51 ('Staurosporine', 'Staurosporine M+H') -507
3312 ('Staurosporine', 'Staurosporine M+H') -457
3096 ('Staurosporine', 'Staurosporine M+H') -456
7857 ('Staurosporine', 'Staurosporine M+H') 90
3096 ('Staurosporine', 'Staurosporine M+H') -456
3 ('Staurosporine', 'Staurosporine M+H') -528
2196 ('Staurosporine', 'Staurosporine M+H') -74
51 ('Staurosporine', 'Staurosporine M+H') -66
3312 ('Staurosporine', 'Staurosporine M+H') 5
3096 ('Staurosporine', 'Staurosporine M+H') 6
7857 ('Staurosporine', 'Staurosporine M+H') 69
3096 ('Staurosporine', 'Staurosporine M+H') 6
3 ('Staurosporine', 'Staurosporine M+H') -66


In [161]:
from genomics import load_mibig_library_json
mibig = load_mibig_library_json('/srv/data/mibig/mibig_json_1.4')
mibig_compound_dict = {}
for b in mibig:
    compound = mibig[b]['general_params']['compounds'][0]['compound']
    mibig_compound_dict[compound] = b
print mibig[b].keys()
compounds = []
for s in spectra:
    if len(s.annotations) > 0:
        for compound,_ in s.annotations:
            compounds.append(compound)
compounds = set(compounds)
for c in compounds:
    if 'derep' in c:
        initial = c.split(';')[0].split('_')[0].lower()
        for co in mibig_compound_dict:
            if initial in co:
                print co,initial,c
        

 Found 1816 files
[u'personal', u'version', u'general_params']
aurantinins B-D aurantin Aurantin_II_24051_dereplictor_pv_9.12748e-26
actinomycin actinomycin Actinomycin_X2;_Actinomycin_V;_Actinomyc_28731_dereplictor_pv_1.70529e-21


-66
